### Udacity Data Engineering Nanodegree
# Project 2: Data Modeling with Apache Cassandra

##### A complete description of the project goals and scope are available [here in the README.md](https://github.com/tommytracey/udacity_data_engineering/tree/master/p2_data_modeling_cassandra)

&nbsp;

#### Import Python packages 

In [1]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

---
# Part I. &mdash; ETL Pipeline
The following code is used for pre-processing the source CSV files

#### Create list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Process the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiate an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# read csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # create a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extract each data row one by one and append it        
        for line in csvreader:
            full_data_rows_list.append(line) 
            
# display total number of rows 
print('total rows from CSV source files: {}'.format(len(full_data_rows_list)))

# preview sample of 3 event data rows
print('\nsource data sample:\n{}'.format(full_data_rows_list[0:3]))

# create an event data csv file - used later to insert data into the Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

total rows from CSV source files: 8056

source data sample:
[['Rokia TraorÃ\x83Â©', 'Logged In', 'Stefany', 'F', '0', 'White', '274.88608', 'free', 'Lubbock, TX', 'PUT', 'NextSong', '1.54071E+12', '693', 'Zen', '200', '1.54251E+12', '83'], ['', 'Logged In', 'Tucker', 'M', '0', 'Garrison', '', 'free', 'Oxnard-Thousand Oaks-Ventura, CA', 'GET', 'Home', '1.54083E+12', '555', '', '200', '1.54251E+12', '40'], ['Camila', 'Logged In', 'Tucker', 'M', '1', 'Garrison', '230.81751', 'free', 'Oxnard-Thousand Oaks-Ventura, CA', 'PUT', 'NextSong', '1.54083E+12', '555', 'Abrazame  (Version Acustica)', '200', '1.54251E+12', '40']]


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print('rows inserted into new CSV file: {}'.format(sum(1 for line in f)))

rows inserted into new CSV file: 6821


#### The new file `event_datafile_new.csv` is located within the Workspace directory. The file contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

Here is a screenshot of what the denormalized data looks like after running the code above.

<img src="images/image_event_datafile_new.jpg">

---
# Part II. Implement Apache Cassandra Database

#### Create a Cassandra cluster

In [5]:
# make a connection to a Cassandra instance your local machine 

from cassandra.cluster import Cluster
try: 
    cluster = Cluster(['127.0.0.1']) #If you have a locally installed Apache Cassandra instance
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [6]:
# create a keyspace 

try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkifydb 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)


#### Set Keyspace

In [7]:
# set KEYSPACE to the keyspace specified above

try:
    session.set_keyspace('sparkifydb')
except Exception as e:
    print(e)

#### Next, we create tables to run the following queries. With Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

##### 1. Give the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


##### 2. Give only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

##### 3. Give every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




### Query 1
**Requested information:** 
Give the artist, song title and song's length in the music app history that was heard during sessionId = 338 and itemInSession = 4

**Approach:**
Since we're looking for a specific item within a specific session, we'll use `session_id` as the partition key and `item_in_session` as the clustering column. From the query results, we'll only display `artist_name`, `song_title`, and `song_duration` within a dataframe. 

In [8]:
## create table

query = "CREATE TABLE IF NOT EXISTS song_in_session "
query = query + "(session_id INT, item_in_session INT, artist_name VARCHAR, song_title VARCHAR, song_duration DECIMAL, \
PRIMARY KEY (session_id, item_in_session))"

try:
    session.execute(query)
except Exception as e:
    print(e)               

In [9]:
## insert data

# read data from CSV
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        # insert line into Cassandra table
        query = "INSERT INTO song_in_session (session_id, item_in_session, artist_name, song_title, song_duration)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

In [10]:
## output the requested information -- artist_name, song_title, song_duration

# run query and save results as dataframe
query = "SELECT * FROM song_in_session WHERE session_id=338 AND item_in_session=4"
try:
    df = pd.DataFrame(list(session.execute(query)))
except Exception as e:
    print(e)
    
# display requested fields
df[['artist_name', 'song_title', 'song_duration']].style.hide_index()

artist_name,song_title,song_duration
Faithless,Music Matters (Mark Knight Dub),495.3073


### Query 2
**Requested information:** 
Give only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

**Approach:**
Since we're seeking results for a specific user and session, we'll use both `user_id` and `session_id` as a [composite partition key](https://docs.datastax.com/en/archived/cql/3.3/cql/cql_using/useCompositePartitionKeyConcept.html). And, because we need to sort the results by `item_in_session`, we'll use that as the clustering column. From the query results, we'll only display `artist_name`, `song_title`, `user_first_name` and `user_last_name` within a dataframe. 

In [11]:
## create table

query = "CREATE TABLE IF NOT EXISTS song_in_user_session "
query = query + "(user_id INT, session_id INT, item_in_session INT, artist_name VARCHAR, song_title VARCHAR, user_first_name VARCHAR, \
user_last_name VARCHAR, PRIMARY KEY ((user_id, session_id), item_in_session))"  # note the composite partition key

try:
    session.execute(query)
except Exception as e:
    print(e)               

In [12]:
## insert data

# read data from CSV
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        # insert line into Cassandra table        
        query = "INSERT INTO song_in_user_session (user_id, session_id, item_in_session, artist_name, song_title, user_first_name, user_last_name)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

In [13]:
## verify the data was inserted correctly

# run query
query = "SELECT * FROM song_in_user_session WHERE user_id=10 AND session_id=182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

# verify data are sorted correctly
print('user_id  session_id  item_in_session')
for row in rows:
    print ('{}       {}         {}'.format(row.user_id, row.session_id, row.item_in_session))

user_id  session_id  item_in_session
10       182         0
10       182         1
10       182         2
10       182         3


In [14]:
## output the requested information -- artist_name, song_title, user_full_name

# run query and save results as dataframe
query = "SELECT * FROM song_in_user_session WHERE user_id=10 AND session_id=182"
try:
    df = pd.DataFrame(list(session.execute(query)))
except Exception as e:
    print(e)

# display requested fields
df[['artist_name', 'song_title', 'user_first_name', 'user_last_name']].style.hide_index()

artist_name,song_title,user_first_name,user_last_name
Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
Three Drives,Greece 2000,Sylvie,Cruz
Sebastien Tellier,Kilometer,Sylvie,Cruz
Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio Edit),Sylvie,Cruz


### Query 3
**Requested information:** 
Give every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

**Approach:**
Since we want the results to be unique based on a specific song, we'll use `song_title` as the partition key. And since we're looking for the set of users who listened to that song, we'll use `user_id` as the clustering column. From the query results, we'll only display `user_first_name` and `user_last_name` within a dataframe. 

In [15]:
## create table

query = "CREATE TABLE IF NOT EXISTS user_song_listen "
query = query + "(user_id INT, user_first_name VARCHAR, user_last_name VARCHAR, song_title VARCHAR, \
PRIMARY KEY (song_title, user_id))"

try:
    session.execute(query)
except Exception as e:
    print(e)               

In [16]:
## insert data

# read data from CSV
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        # insert line into Cassandra table
        query = "INSERT INTO user_song_listen (user_id, user_first_name, user_last_name, song_title)"
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, (int(line[10]), line[1], line[4], line[9]))

In [17]:
## output the requested information -- user_first_name, user_last_name

# run query and save results as dataframe
query = "SELECT * FROM user_song_listen WHERE song_title='All Hands Against His Own'"
try:
    df = pd.DataFrame(list(session.execute(query)))
except Exception as e:
    print(e)

# display requested fields
df[['user_first_name', 'user_last_name']].style.hide_index()

user_first_name,user_last_name
Jacqueline,Lynch
Tegan,Levine
Sara,Johnson


### Drop the tables before closing out the sessions

In [18]:
query1 = "drop table song_in_session"
query2 = "drop table song_in_user_session"
query3 = "drop table user_song_listen"

try:
    rows1 = session.execute(query1)
    rows2 = session.execute(query2)
    rows3 = session.execute(query3)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [19]:
session.shutdown()
cluster.shutdown()